In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from torch.utils.data import DataLoader
from torchsummary import summary
from IPython.core.debugger import set_trace

from Tools import Trainer
from Modules.Residual import ResidualConv

## Loading datasets

In [2]:
from Tools.Dataloader import datasets_loader
train_loader, valid_loader, test_loader = datasets_loader()

## Hyper-parameters

In [3]:
hyperparams = {'num_epochs': 10,
               'lr0' :0.1, 
               'save_checkpoint': False
                  }
cuda_available = torch.cuda.is_available()


## Residual Classifier

In [4]:
class ResidualClassifier(nn.Module):
    def __init__(self):
        super(ResidualClassifier, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            ResidualConv(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1),
#             nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            ResidualConv(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
#             nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            ResidualConv(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
#             nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            )
        # Logistic Regression
#         def classif
        self.clf = nn.Linear(8*8*64, 2)

    def forward(self,x):
        x = self.conv(x)
        x = x.view(-1, 8*8*64)
        x = self.clf(x)
        return x
#     self.clf(self.conv(x).squeeze())

In [5]:
model = ResidualClassifier()

if cuda_available:
    model = model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=hyperparams['lr0'])
criterion = nn.CrossEntropyLoss()

In [6]:
print(model)

ResidualClassifier(
  (conv): Sequential(
    (0): ResidualConv(
      (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
      (relu1): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
      (relu2): ReLU(inplace)
      (projected): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (1): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ResidualConv(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
      (relu1): ReLU(inplace)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
      (relu2): ReLU(inplace)
      (projected): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ResidualConv(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
      (relu1): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_

In [9]:
# summary(model, (1, 8, 8))

In [13]:
trainer = Trainer(model, optimizer, criterion, train_loader, valid_loader, test_loader, hyperparams)
nll_train, nll_test, acc_train, acc_test =  trainer.train_model()
print('Done...')

TypeError: 'module' object is not callable